In [2]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [3]:
import dash
from dash import dcc, html, Input, Output
import yfinance as yf
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import plotly.graph_objects as go



In [4]:
# Initialize Dash app
app = dash.Dash(__name__)
server = app.server

In [5]:
# Define layout
app.layout = html.Div([
    html.H1("Stock Price Visualizer and Forecaster", style={'textAlign': 'center'}),

    html.Div([
        html.Label("Enter Stock Ticker (e.g., AAPL, TSLA):"),
        dcc.Input(id='stock-ticker', value='AAPL', type='text', debounce=True),
        html.Button('Submit', id='submit-button', n_clicks=0)
    ], style={'textAlign': 'center', 'marginBottom': 20}),

    dcc.Graph(id='stock-graph'),
    dcc.Graph(id='forecast-graph'),

    html.Div(id='model-performance', style={'textAlign': 'center', 'marginTop': 20})
])



In [7]:
@app.callback(
    [Output('stock-graph', 'figure'),
     Output('forecast-graph', 'figure'),
     Output('model-performance', 'children')],
    [Input('submit-button', 'n_clicks')],
    [dash.dependencies.State('stock-ticker', 'value')]
)
def update_graph(n_clicks, ticker):
    if not ticker:
        return {}, {}, "Please enter a valid stock ticker."

    # Fetch historical stock data
    try:
        stock_data = yf.download(ticker, period='1y')
        stock_data.reset_index(inplace=True)
    except Exception as e:
        return {}, {}, f"Error fetching data: {e}"

    # Plot historical stock prices
    stock_fig = go.Figure()
    stock_fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['Close'], mode='lines', name='Close Price'))
    stock_fig.update_layout(title=f"{ticker.upper()} Stock Prices (Last 1 Year)", xaxis_title='Date', yaxis_title='Price (USD)')

    # Prepare data for forecasting
    stock_data['DateOrdinal'] = stock_data['Date'].map(pd.Timestamp.toordinal)
    X = stock_data[['DateOrdinal']].values
    y = stock_data['Close'].values

    # Train SVR model
    svr = SVR()
    param_grid = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1], 'kernel': ['rbf']}
    grid_search = GridSearchCV(svr, param_grid, cv=3)
    grid_search.fit(X, y)
    best_model = grid_search.best_estimator_

    # Predict future prices
    future_dates = pd.date_range(stock_data['Date'].iloc[-1], periods=30, freq='B').to_list()
    future_ordinals = np.array([date.toordinal() for date in future_dates]).reshape(-1, 1)
    predictions = best_model.predict(future_ordinals)

    # Plot forecasted stock prices
    forecast_fig = go.Figure()
    forecast_fig.add_trace(go.Scatter(x=future_dates, y=predictions, mode='lines', name='Forecasted Price'))
    forecast_fig.update_layout(title=f"{ticker.upper()} Stock Price Forecast (Next 30 Days)",
                                xaxis_title='Date', yaxis_title='Price (USD)')

    # Calculate model performance
    mse = mean_squared_error(y, best_model.predict(X))
    mae = mean_absolute_error(y, best_model.predict(X))
    performance_text = f"Model Performance: MSE = {mse:.2f}, MAE = {mae:.2f}"

    return stock_fig, forecast_fig, performance_text

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>